In [3]:
from dft2cc.utils.mol import Mol
import pyscf
import numpy as np
from dft2cc.utils.grids import rotate

MASS = {
    'H': 1.00782503207,
    'He': 4.00260325415,
    'Li': 6.938,
    'Be': 9.012183065,
    'B': 10.806,
    'C': 12.0096,
    'N': 14.006855,
    'O': 15.9994,
    'F': 18.998403163,
    'Ne': 20.1797,
}

def get_barycenter(molecular):
    barycenter = np.array([0, 0, 0], dtype=np.float64)
    mass = 0.0
    for mol in molecular:
        mass += MASS[mol[0]]
        barycenter[0] += mol[1] * MASS[mol[0]]
        barycenter[1] += mol[2] * MASS[mol[0]]
        barycenter[2] += mol[3] * MASS[mol[0]]
    return barycenter / mass

def get_inertia_moment(molecular):
    # Get the moment of inertia
    I = np.zeros((3, 3), dtype=np.float64)
    for mol in molecular:
        I[0, 0] += MASS[mol[0]] * (mol[2] ** 2 + mol[3] ** 2)
        I[1, 1] += MASS[mol[0]] * (mol[1] ** 2 + mol[3] ** 2)
        I[2, 2] += MASS[mol[0]] * (mol[1] ** 2 + mol[2] ** 2)
        I[0, 1] -= MASS[mol[0]] * mol[1] * mol[2]
        I[0, 2] -= MASS[mol[0]] * mol[1] * mol[3]
        I[1, 2] -= MASS[mol[0]] * mol[2] * mol[3]
        I[1, 0] = I[0, 1]
        I[2, 0] = I[0, 2]
        I[2, 1] = I[1, 2]
    return I


def rotation_matrix_from_vectors(vec1, vec2):
    """Find the rotation matrix that aligns vec1 to vec2
    :param vec1: A 3d "source" vector
    :param vec2: A 3d "destination" vector
    :return mat: A transform matrix (3x3) which when applied to vec1, aligns it with vec2.
    """
    a, b = (vec1 / np.linalg.norm(vec1)).reshape(3), (
        vec2 / np.linalg.norm(vec2)
    ).reshape(3)
    v = np.cross(a, b)
    c = np.dot(a, b)
    s = np.linalg.norm(v)
    kmat = np.array([[0, -v[2], v[1]], [v[2], 0, -v[0]], [-v[1], v[0], 0]])
    rotation_matrix = np.eye(3) + kmat + kmat.dot(kmat) * ((1 - c) / (s**2))
    return rotation_matrix

def my_print(molecular):
    for mol in molecular:
        print(f"{mol[0]} {mol[1]:.3f}, {mol[2]:.3f}, {mol[3]:.3f}")

# molecular = Mol["HH"]
# molecular[0][1] = 0.5

# rotate(molecular)
# print(molecular)

In [19]:
import copy
for i in range(100):
    molecular = copy.deepcopy(Mol["HOH"])
    molecular[0][1] = 0.5
    rotate(molecular, angle_list="random")

    # Get the barycenter
    barycenter = get_barycenter(molecular)

    for mol in molecular:
        mol[1] -= barycenter[0]
        mol[2] -= barycenter[1]
        mol[3] -= barycenter[2]

    I = get_inertia_moment(molecular)
    eig_val, eig_vec = np.linalg.eig(I)
    index1 = np.argsort(eig_val)[2]
    index2 = np.argsort(eig_val)[1]
    if np.abs((eig_val[index1] - eig_val[index2])) > 1e-12:
        print("###")
        list_max_eig = eig_vec[:, index1]
        rotation_matrix = rotation_matrix_from_vectors(list_max_eig, [0, 0, 1])
        # rotate the molecule
        for mol in molecular:
            x_array = np.array(mol[1:])
            x_array = rotation_matrix @ x_array
            mol[1] = x_array[0]
            mol[2] = x_array[1]
            mol[3] = x_array[2]

        I = get_inertia_moment(molecular)
        eig_val, eig_vec = np.linalg.eig(I)
        index2 = np.argsort(eig_val)[1]
        list_max_eig = eig_vec[:, index2]
        rotation_matrix = rotation_matrix_from_vectors(list_max_eig, [0, 1, 0])
        for mol in molecular:
            x_array = np.array(mol[1:])
            x_array = rotation_matrix @ x_array
            mol[1] = x_array[0]
            mol[2] = x_array[1]
            mol[3] = x_array[2]
    else:
        index1 = np.argsort(eig_val)[0]
        list_max_eig = eig_vec[:, index1]

        if (np.sqrt(list_max_eig[1] ** 2 + list_max_eig[2] ** 2)) > 1e-6:
            rotation_matrix = rotation_matrix_from_vectors(list_max_eig, [1, 0, 0])

            # rotate the molecule
            for mol in molecular:
                x_array = np.array(mol[1:])
                x_array = rotation_matrix @  x_array
                mol[1] = x_array[0]
                mol[2] = x_array[1]
                mol[3] = x_array[2]

        for mol in molecular:
            x_array = np.array(mol[1:])
            index_ = np.argsort(np.abs(x_array))[-1]
            mol[1] = x_array[index_]

    if molecular[0][1] > 0:
        for i_mol, mol in enumerate(molecular):
            mol[1] = -mol[1]
    if molecular[0][2] > 0:
        for i_mol, mol in enumerate(molecular):
            mol[2] = -mol[2]
    if molecular[0][3] > 0:
        for i_mol, mol in enumerate(molecular):
            mol[3] = -mol[3]
    my_print(molecular)
    print("=============")

###
H -0.220, -0.436, -0.000
O -0.046, 0.032, 0.000
H 0.948, -0.076, -0.000
###
H -0.220, -0.436, -0.000
O -0.046, 0.032, -0.000
H 0.948, -0.076, 0.000
###
H -0.220, -0.436, -0.000
O -0.046, 0.032, -0.000
H 0.948, -0.076, 0.000
###
H -0.220, -0.436, -0.000
O -0.046, 0.032, 0.000
H 0.948, -0.076, -0.000
###
H -0.220, -0.436, -0.000
O -0.046, 0.032, -0.000
H 0.948, -0.076, 0.000
###
H -0.220, -0.436, -0.000
O -0.046, 0.032, -0.000
H 0.948, -0.076, 0.000
###
H -0.220, -0.436, -0.000
O -0.046, 0.032, -0.000
H 0.948, -0.076, 0.000
###
H -0.220, -0.436, -0.000
O -0.046, 0.032, 0.000
H 0.948, -0.076, -0.000
###
H -0.220, -0.436, -0.000
O -0.046, 0.032, -0.000
H 0.948, -0.076, 0.000
###
H -0.220, -0.436, -0.000
O -0.046, 0.032, 0.000
H 0.948, -0.076, 0.000
###
H -0.220, -0.436, -0.000
O -0.046, 0.032, 0.000
H 0.948, -0.076, -0.000
###
H -0.220, -0.436, -0.000
O -0.046, 0.032, 0.000
H 0.948, -0.076, -0.000
###
H -0.220, -0.436, -0.000
O -0.046, 0.032, -0.000
H 0.948, -0.076, 0.000
###
H -0.220,

In [23]:
# Get the moment of inertia
I = np.zeros((3, 3), dtype=np.float64)
for i_mol, mol in enumerate(molecular):
    I[0, 0] += MASS[mol[0]] * (mol[2] ** 2 + mol[3] ** 2)
    I[1, 1] += MASS[mol[0]] * (mol[1] ** 2 + mol[3] ** 2)
    I[2, 2] += MASS[mol[0]] * (mol[1] ** 2 + mol[2] ** 2)
    I[0, 1] -= MASS[mol[0]] * mol[1] * mol[2]
    I[0, 2] -= MASS[mol[0]] * mol[1] * mol[3]
    I[1, 2] -= MASS[mol[0]] * mol[2] * mol[3]
    I[1, 0] = I[0, 1]
    I[2, 0] = I[0, 2]
    I[2, 1] = I[1, 2]


# Get the barycenter
barycenter = np.array([0, 0, 0], dtype=np.float64)
for i_mol, mol in enumerate(molecular):
    barycenter[0] += mol[1] * MASS[mol[0]]
    barycenter[1] += mol[2] * MASS[mol[0]]
    barycenter[2] += mol[3] * MASS[mol[0]]
barycenter = barycenter / np.sum([MASS[mol[0]] for mol in molecular])

print(molecular, "\n", "\n", I, "\n", "\n", barycenter)

[['H', 5.551115123125783e-17, -3.3990776836172276e-33, 0.7071067811865475], ['H', -5.551115123125783e-17, 3.3990776836172276e-33, -0.7071067811865475]] 
 
 [[ 1.00782503e+00  3.80326390e-49 -7.91189241e-17]
 [ 3.80326390e-49  1.00782503e+00  4.84463686e-33]
 [-7.91189241e-17  4.84463686e-33  6.21120131e-33]] 
 
 [0. 0. 0.]
